In [ ]:
import numpy as np
import os
import sys
import xarray as xr

sys.path.append("../")
from  STORM.preprocessing import extract_data
from matplotlib.collections import LineCollection
import cmocean

## set matplotlib fonts
font_size = 48
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams['axes.labelsize'] = font_size
plt.rcParams['axes.titlesize'] = font_size
plt.rcParams['xtick.labelsize'] = font_size
plt.rcParams['ytick.labelsize'] = font_size
plt.rcParams['legend.fontsize'] = font_size
plt.rcParams['legend.title_fontsize'] = font_size


In [1]:

## RUN to extract dataset, for preprocessing

data=xr.open_dataset('../STORM/IBTrACS.since1980.v04r01.nc',decode_times=False)
extract_data(data, save_path="../STORM/")
data.close()


NameError: name 'xr' is not defined

In [ ]:
## load preprocessed tropical cyclone data

latlist = np.load("../STORM/LATLIST_INTERP.npy", allow_pickle=True).item()
lonlist = np.load("../STORM/LONLIST_INTERP.npy", allow_pickle=True).item()
windlist = np.load("../STORM/WINDLIST_INTERP.npy", allow_pickle=True).item()
rmaxlist = np.load("../STORM/RMAXLIST_INTERP.npy", allow_pickle=True).item()
yearlist = np.load("../STORM/YEARLIST_INTERP.npy", allow_pickle=True).item()
monthlist = np.load("../STORM/MONTHLIST_INTERP.npy", allow_pickle=True).item()

In [ ]:
## reform data to be ready to display it

data = {"lat": [], "lon": [], "wind": []}

for storm_index in latlist.keys():
    if storm_index > 2: break
    lats = latlist[storm_index]
    if len(lats) == 0: continue
    lons = lonlist[storm_index]
    winds = windlist[storm_index]
    
    winds = list(np.nan_to_num(winds))
    
    data["lat"] += lats
    data["lon"] += lons
    data["wind"] += winds

In [ ]:

def colored_lines(xs, ys, cs, ax, **lc_kwargs):
    """
    Cite: From MATPLOTLIB documentation
    
    Plot a line with a color specified along the line by a third value.

    It does this by creating a collection of line segments. Each line segment is
    made up of two straight lines each connecting the current (x, y) point to the
    midpoints of the lines connecting the current point with its two neighbors.
    This creates a smooth line with no gaps between the line segments.

    Parameters
    ----------
    x, y : array-like
        The horizontal and vertical coordinates of the data points.
    c : array-like
        The color values, which should be the same size as x and y.
    ax : Axes
        Axis object on which to plot the colored line.
    **lc_kwargs
        Any additional arguments to pass to matplotlib.collections.LineCollection
        constructor. This should not include the array keyword argument because
        that is set to the color argument. If provided, it will be overridden.

    Returns
    -------
    matplotlib.collections.LineCollection
        The generated line collection representing the colored line.
    """
    if "array" in lc_kwargs:
        warnings.warn('The provided "array" keyword argument will be overridden')

    # Default the capstyle to butt so that the line segments smoothly line up
    default_kwargs = {"capstyle": "butt",'transform': ccrs.PlateCarree()}
    default_kwargs.update(lc_kwargs)

    all_segments = []
    all_cs = []
    for i in range(len(xs)):
        x = xs[i]
        if len(x) == 0: continue
        y = ys[i]
        c = cs[i]
        all_cs.append(c)
        # Compute the midpoints of the line segments. Include the first and last points
        # twice so we don't need any special syntax later to handle them.
        x = np.asarray(x)
        y = np.asarray(y)
    
        x_midpts = np.hstack((x[0], 0.5 * (x[1:] + x[:-1]), x[-1]))
        y_midpts = np.hstack((y[0], 0.5 * (y[1:] + y[:-1]), y[-1]))
    
        # Determine the start, middle, and end coordinate pair of each line segment.
        # Use the reshape to add an extra dimension so each pair of points is in its
        # own list. Then concatenate them to create:
        # [
        #   [(x1_start, y1_start), (x1_mid, y1_mid), (x1_end, y1_end)],
        #   [(x2_start, y2_start), (x2_mid, y2_mid), (x2_end, y2_end)],
        #   ...
        # ]
        coord_start = np.column_stack((x_midpts[:-1], y_midpts[:-1]))[:, np.newaxis, :]
        coord_mid = np.column_stack((x, y))[:, np.newaxis, :]
        coord_end = np.column_stack((x_midpts[1:], y_midpts[1:]))[:, np.newaxis, :]
        segments = np.concatenate((coord_start, coord_mid, coord_end), axis=1)
        all_segments.append(segments)
    all_segments = np.concatenate(all_segments, axis=0)
    all_cs = np.concatenate(all_cs, axis=0)
    print(np.max(all_cs))
    lc = LineCollection(all_segments, **default_kwargs)
    lc.set_array(all_cs)  # set the colors of each segment

    return ax.add_collection(lc)
    
def plot_IBTrACS(latlist, lonlist, windlist):
    """
        Makes a plot of the ibtracs storms showing the tracks colored by the maximum wind speed
    """
    fig, axes = plt.subplots(1, figsize=(20,15), layout="compressed", subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})

    # Add transparent land by creating a custom feature
    land_shp = natural_earth(resolution='110m', category='physical', name='land')
    land_feature = ShapelyFeature(Reader(land_shp).geometries(),
                                  ccrs.PlateCarree(), facecolor='lightgray', edgecolor='face', alpha=0.5)
    # Create a feature for States/Admin 1 regions at 1:50m from Natural Earth
    states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')
    
    # Add countries boundaries and labels
    countries_shp = natural_earth(resolution='50m', category='cultural', name='admin_0_countries')
    countries_feature = ShapelyFeature(Reader(countries_shp).geometries(), ccrs.PlateCarree(),
                                       facecolor='none', edgecolor='black', linewidth=1, alpha=1)

    
    ax = axes
    ax.add_feature(land_feature)
    ax.add_feature(countries_feature)
    ax.add_feature(states_provinces, edgecolor='gray')

    ax.set_xticks([])
    ax.set_yticks([])


    # Plot gradient for model prediction on top of the land feature
    ##lonplot2 = np.linspace(135, 240, val.shape[1])
    #latplot2 = np.linspace(-5, -60, val.shape[0])
    
    # Use a continuous colormap
    #pc[category-1][month] = ax.contourf(lonplot2, latplot2, val, levels=100, cmap=cmocean.cm.matter, transform=ccrs.PlateCarree())
    #print(ibtracs_df["lat"].tolist())

    lines = colored_lines(list(lonlist.values()), list(latlist.values()), list(windlist.values()), ax, linewidth=5, cmap=cmocean.cm.thermal)
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False,
                              linewidth=1, color='white', alpha=0.3, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    plt.colorbar(lines)
    ax.set_extent([135, 240, -60, -5], crs=ccrs.PlateCarree())
        
    #fig.subplots_adjust(right=0.8)
    #cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    plt.show()

In [ ]:
# make the plot
plot_IBTrACS(latlist, lonlist, windlist)

In [2]:
### Code to plot the average number of cat 3-5 TCs per decade using only IBTrACS data

from STORM.SAMPLE_TC_PRESSURE import TC_Category
from TrainingDataGeneration.RiskFactors import get_landfalls_data


def create_TC_data(latlist, lonlist, windlist, rmaxlist, yearlist, monthlist):
    """
    takes TC data loaded from IBTrACS and converts it into the format expected by get_landfalls data
    
    data format [( 
        year,
        month,
        storm_number,
        l,
        idx,
        lat (one step),
        lon (one),
        pressure (one),
        wind (one),
        storm_radius_max_wind_speeds (one),
        category,
        landfall (one),
        distance)]
    """
    TC_data = []
    
    for storm_index in range(len(latlist.keys())):
        lats = latlist[storm_index]
        if len(lats) == 0: continue
        lons = lonlist[storm_index]
        winds = windlist[storm_index]
        rmaxs = rmaxlist[storm_index]
        
        if np.isnan(np.min(rmaxs)): rmaxs = [46.29999923706055] * len(lats)
        
        
        year = yearlist[storm_index][0] - 1979
        month = monthlist[storm_index][0]
        if month not in [1, 2, 3, 4, 11, 12]: continue
        
        for i in range(len(lats)):
            TC_data.append(
                [year,
                 month,
                 storm_index,
                 i,
                 i,
                 lats[i],
                 lons[i],
                 0,
                 winds[i],
                 rmaxs[i],
                 TC_Category(winds[i]),
                 0,
                 0
                ])
    
    return TC_data

ModuleNotFoundError: No module named 'STORM'

In [ ]:

# convert the TC data
TC_data = create_TC_data(latlist, lonlist, windlist, rmaxlist, yearlist, monthlist)

# calculate the mean TC counts
means, stds = get_landfalls_data(TC_data,
        "SP",
        4,
        0.5,
        False,
        storm_rmax_multiple=4
)


In [ ]:
# adjust the mean value because we don't actually have 4 decades, we have 3.8 decades
n_decades_actual = 3.8

# calculate the mean cat 3-5 storms from the separate means
mean_decadal = np.flipud(np.sum(means[:, :, :, 3:] * 4 / n_decades_actual, axis=(-2,-1)))

In [ ]:
def plot_map_data(data):
    """ 
    Makes a plot showing a 2d matrix data as a contour plot over the south pacific basin
    """
    
    fig, axes = plt.subplots(1, 1, figsize=(20,15), layout="compressed", subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})

    # Add transparent land by creating a custom feature
    land_shp = natural_earth(resolution='110m', category='physical', name='land')
    land_feature = ShapelyFeature(Reader(land_shp).geometries(),
                                  ccrs.PlateCarree(), facecolor='lightgray', edgecolor='face', alpha=0.5)
    # Create a feature for States/Admin 1 regions at 1:50m from Natural Earth
    states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')
    
    # Add countries boundaries and labels
    countries_shp = natural_earth(resolution='50m', category='cultural', name='admin_0_countries')
    countries_feature = ShapelyFeature(Reader(countries_shp).geometries(), ccrs.PlateCarree(),
                                       facecolor='none', edgecolor='black', linewidth=1, alpha=1)


    ax = axes
    ax.add_feature(land_feature)
    ax.add_feature(countries_feature)
    ax.add_feature(states_provinces, edgecolor='gray')

    ax.set_xticks([])
    ax.set_yticks([])


    # Plot gradient for model prediction on top of the land feature
    lonplot2 = np.linspace(135, 240, data.shape[1])
    latplot2 = np.linspace(-5, -60, data.shape[0])
            
    # Use a continuous colormap
    c = ax.contourf(lonplot2, latplot2, data, levels=100, cmap=cmocean.cm.matter, transform=ccrs.PlateCarree())
            
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                        linewidth=1, color='black', alpha=0.3, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    ax.set_extent([135, 240, -60, -5], crs=ccrs.PlateCarree())

        
    gl.xlocator = mticker.FixedLocator([-180, -160, -140, -90, -45, 0, 45, 90, 140, 160, 180, 225, 270, 315, 360])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 48, 'color': 'black'}
    gl.ylabel_style = {'size': 48, 'color': 'black'}
            
    # Adjust map extent to focus on the region of interest
    
    cbar = fig.colorbar(c, ax=ax)
    nbins = 5
    tick_locator = mticker.MaxNLocator(nbins=nbins)
    cbar.locator = tick_locator
    cbar.update_ticks()
    cbar.ax.tick_params(labelsize=48)
    
    plt.show()


In [ ]:
# plot the mean decadal counts

plot_map_data(mean_decadal)